Nice, let’s “lock in” Section 2 as a **pure config consumer** with a clean 2.0A header + a single helper that guarantees everything is ready.

Here’s a drop-in **2.0A block** you can paste at the **top of Section 2** (right before 2.0.1). It does:

* Docstring-style overview for 2.0A.
* A helper `assert_config_ready_for_section2()` that:

  * Checks `CONFIG`, `C`, `PROJECT_ROOT`, `df`, `REPORTS_DIR`, etc.
  * Validates key config roots: `TARGET`, `ID_COLUMNS`, `RANGES`, `DATA_QUALITY`.
* Prints a short “OK, Section 2 may proceed” summary.

---

### 🧩 2.0A Environment & Config Readiness (drop-in code)

```python
# ════════════════════════════════════════════════════════════════════════════════
# 2.0A ⚙️ Environment & Path Initialization (Section 2 preflight)
# 
# Purpose:
#   - Assert that Section 1 has fully run (config, paths, data load).
#   - Confirm key config roots for Section 2 are present.
#   - Ensure report/artifact directories exist and are writable.
#
# This cell does NOT load config. It only CONSUMES:
#   - PROJECT_ROOT, CONFIG, C(), df
#   - REPORTS_DIR, ARTIFACTS_DIR, FIGURES_DIR, etc.
#
# If everything is OK, later sections (2.0.1–2.9) can assume:
#   - df is a valid working DataFrame
#   - CONFIG has the required keys for Section 2
#   - paths & directories are ready for reports and artifacts
# ════════════════════════════════════════════════════════════════════════════════

from pathlib import Path

def assert_config_ready_for_section2() -> None:
    """
    Preflight checks for Section 2.

    Raises informative exceptions if:
      - Section 1 was not run (missing CONFIG, C, df, PROJECT_ROOT, paths)
      - Required CONFIG roots for Section 2 are missing
    """
    # --- 1. Global objects from Section 1 ---------------------------------------
    required_globals = [
        "PROJECT_ROOT",
        "CONFIG",
        "C",
        "RAW_DATA",
        "REPORTS_DIR",
        "ARTIFACTS_DIR",
        "FIGURES_DIR",
        "MODELS_DIR",
        "OUTPUTS_DIR",
        "df",
    ]

    missing = [g for g in required_globals if g not in globals()]
    if missing:
        raise RuntimeError(
            "❌ Section 2 preflight failed — missing globals from Section 1: "
            + ", ".join(missing)
        )

    if not callable(C):
        raise TypeError("❌ C is not callable. Did Section 1.3 run correctly?")

    # --- 2. Basic path sanity ---------------------------------------------------
    core_dirs = {
        "REPORTS_DIR": REPORTS_DIR,
        "ARTIFACTS_DIR": ARTIFACTS_DIR,
        "FIGURES_DIR": FIGURES_DIR,
        "MODELS_DIR": MODELS_DIR,
        "OUTPUTS_DIR": OUTPUTS_DIR,
    }

    for name, d in core_dirs.items():
        if not isinstance(d, Path):
            raise TypeError(f"❌ {name} is not a pathlib.Path: {d!r}")
        d.mkdir(parents=True, exist_ok=True)

    # --- 3. Required CONFIG roots for Section 2 --------------------------------
    required_roots = ["TARGET", "ID_COLUMNS", "RANGES", "DATA_QUALITY"]

    missing_roots = [r for r in required_roots if r not in CONFIG]
    if missing_roots:
        raise KeyError(
            "❌ Section 2 requires config roots missing from CONFIG: "
            + ", ".join(missing_roots)
        )

    # Optional: soft-check flags block
    if "FLAGS" not in CONFIG:
        print("⚠️ CONFIG.FLAGS missing (optional). Using defaults in code where needed.")

    # --- 4. DataFrame sanity ----------------------------------------------------
    if "df" not in globals():
        raise NameError("❌ Working DataFrame 'df' not found. Run Section 1.5 first.")

    n_rows, n_cols = df.shape
    if n_rows == 0 or n_cols == 0:
        raise ValueError(f"❌ 'df' is empty, shape={df.shape}. Section 2 cannot proceed.")

    print("✅ Section 2 preflight OK.")
    print(f"   • df shape: {n_rows:,} rows × {n_cols:,} columns")
    print(f"   • PROJECT_ROOT: {PROJECT_ROOT}")
    print(f"   • REPORTS_DIR:  {REPORTS_DIR}")
    print(f"   • Required CONFIG roots present: {', '.join(required_roots)}")


# --- Run the preflight once at the top of Section 2 ----------------------------
print("📋 2.0A ⚙️ Environment & Config Readiness Check")
assert_config_ready_for_section2()
```

---

### How this fits your 2.0 chain

With this in place, your Section 2 now flows like:

1. **2.0A** – Preflight

   * `assert_config_ready_for_section2()`
   * Confirms Type 3 (YAML) → Type 2 (CONFIG) bridge is ready.

2. **2.0.1 Reporting Bootstrap/Setup**

   * Create `/resources/reports/section2/`
   * Build unified `section2_report_path`, manifest, etc.

3. **2.0.2 Config & Constants Registration**

   * Use the `CONFIG`/`C()`/`target_name`/`id_cols` you already have.
   * Build `section2_config_checks.csv` and append summary to the unified Section 2 report.

4. **2.0.3 Logging & Metadata Setup**

   * Create `section2.log` / `section2_run_metadata.json` in `ARTIFACTS_DIR`.

You’ve now made it explicit in code that:

* **Section 1 = Type 3 → Type 2 layer (load & resolve)**
* **Section 2 = consumer of config + reporter of environment health**

If you’d like, next step I can:

* Add a small **2.0.3 Logging & Metadata Setup** block that writes a `section2_run_metadata.json` with:

  * timestamp
  * git hash (if available)
  * script/notebook name
  * dataset version_id from your `dataset_version_registry.csv`
    so each Section 2 run is auditable.




># Section 2: T3_L4 functions
---

># Section 2.0.3 T3_L4 functions
---

In [ ]:
# 2.0.3 🧾 Logging & Metadata Setup (Function: Use with 2.0A)
# Purpose:
#   Create an auditable metadata record for each Section 2 run.
#   Captures: timestamp, git hash (if available), script/notebook name,
#   dataset version_id (from dataset_version_registry.csv).
# ════════════════════════════════════════════════════════════════════════════════

print("📋 2.0.3 🧾 Logging & Metadata Setup")

from datetime import datetime
import json
import subprocess
import os

# --- 1️⃣ Timestamp ---------------------------------------------------------------
run_ts = datetime.utcnow().isoformat(timespec="seconds") + "Z"

# --- 2️⃣ Git hash (if available) --------------------------------------------------
try:
    git_hash = (
        subprocess.check_output(["git", "rev-parse", "HEAD"], cwd=PROJECT_ROOT)
        .decode("utf-8")
        .strip()
    )
except Exception:
    git_hash = None
    print("⚠️  Git hash unavailable (not a repo or no git installed).")

# --- 3️⃣ Notebook or script name -------------------------------------------------
try:
    script_name = Path(__file__).name  # works if .py
except NameError:
    # Fallback for Jupyter notebooks
    import ipynbname
    try:
        script_name = ipynbname.name() + ".ipynb"
    except Exception:
        script_name = "unknown_notebook_or_script"

# --- 4️⃣ Dataset version lookup --------------------------------------------------
version_registry_path = ARTIFACTS_DIR / "dataset_version_registry.csv"
version_id = None

if version_registry_path.exists():
    try:
        reg_df = pd.read_csv(version_registry_path)
        if not reg_df.empty:
            version_id = str(reg_df["version_id"].iloc[-1])
    except Exception as e:
        print(f"⚠️  Could not read dataset version registry: {e}")
else:
    print(f"⚠️  Dataset version registry not found: {version_registry_path}")

# --- 5️⃣ Compose metadata record -------------------------------------------------
section2_run_metadata = {
    "timestamp_utc": run_ts,
    "git_hash": git_hash,
    "script_or_notebook": script_name,
    "dataset_version_id": version_id,
    "project_root": str(PROJECT_ROOT),
    "reports_dir": str(REPORTS_DIR),
    "user": os.getenv("USER") or os.getenv("USERNAME"),
}

# --- 6️⃣ Save metadata snapshot --------------------------------------------------
metadata_path = ARTIFACTS_DIR / "section2_run_metadata.json"

with open(metadata_path, "w", encoding="utf-8") as f:
    json.dump(section2_run_metadata, f, indent=2)

print(f"✅ Section 2 run metadata written → {metadata_path}")
print(json.dumps(section2_run_metadata, indent=2))
